In [1]:
import math

import nbimporter
import s11_external_condition as EC
import s11_02_solar_radiation as SR
import Loads_and_Water_Temp as LWT

import matplotlib.pyplot as plt
%matplotlib inline

Importing Jupyter notebook from s11_external_condition.ipynb
Importing Jupyter notebook from s11_02_solar_radiation.ipynb


ModuleNotFoundError: No module named 'Loads_and_Water_Temp'

# 第九章　自然エネルギー利用設備

## 第三節　空気集熱式太陽熱利用設備

### 5. 給湯部における補正集熱量

日付$d$における1日当たりの給湯部における補正集熱量$L_{sun,ass,d}$

$$
\begin{equation*}
    L_{sun,ass,d} = min \left( L_{tunk,d} , \: \left( L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} \right) \times f_{sr,uplim} \right)
\end{equation*}
$$

$f_{sr,uplim}$ : 給湯部の分担率上限値(-)   
$L_{sun,ass,d}$ : 日付$d$における1日当たりの給湯部における補正集熱量(MJ/日)  
$L_{tnk,d}$ : 日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量(MJ/日)  
$L'_{k,d}$ : 日付$d$における1日当たりの台所水栓における節湯補正給湯熱負荷(MJ/日)  
$L'_{s,d}$ : 日付$d$における1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/日)  
$L'_{w,d}$ : 日付$d$における1日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/日)  
$L'_{b1,d}$ : 日付$d$における1日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/日)  
$L'_{b2,d}$ : 日付$d$における1日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/日)  
$L'_{ba1,d}$ : 日付$d$における1日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/日)

給湯部の分担率上限値$f_{sr,uplim}$

$$
\begin{equation*}
    f_{sr,uplim} = 0.9
\end{equation*}
$$

In [ ]:
def get_L_sun_ass(L_tnk, L_dash_s):
    f_sr_uplim = 0.9 # -
    return min(L_tnk, sum(L_dash_s) * f_sr_uplim)

日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量$L_{tnk,d}$

$$
\begin{equation*}
   L_{tnk,d} = min \left( Q_{d} , \: HC_{tnk,d} \times \alpha_{tnk,d} \right)
\end{equation*}
$$

$L_{tnk,d}$ : 日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量(MJ/日)  
$Q_{d}$ : 日付$d$における1日当たりの基準集熱量(MJ/日)  
$HC_{tnk,d}$ : 日付$d$における給湯部のタンク蓄熱量の上限(MJ)  
$\alpha_{tnk,d}$ : 日付$d$における給湯部のタンク有効利用率(1/日)  

日付$d$における給湯部のタンク有効利用率$\alpha_{tnk,d}$

$$
\begin{equation*}
    \alpha_{tnk,d} = 1.0
\end{equation*}
$$

In [ ]:
def get_L_tnk(Q_d, HC_tnk):
    alpha_tnk = 1  # 1/day
    return min(Q_d, HC_tnk * alpha_tnk)

日付$d$における給湯部のタンク蓄熱量の上限$HC_{tnk,d}$

$$
\begin{equation*}
    HC_{tnk,d} = \left( \theta_{tnk} - \theta_{wtr,d} \right) \times W_{tnk} \times 10^{-3}
\end{equation*}
$$

$HC_{tnk,d}$ : 日付$d$における給湯部のタンク蓄熱量の上限(MJ)  
$\theta_{tnk}$ : 給湯部のタンク内温度(℃)   
$\theta_{wtr,d}$ : 日付$d$における日平均給水温度(℃)  
$W_{tnk}$ : 給湯部のタンク容量(L)  

給湯部のタンク内温度$\theta_{tnk}$

$$
\begin{equation*}
    \theta_{tnk} = 65
\end{equation*}
$$

In [ ]:
def get_HC_tnk(theta_wtr, W_tnk):
    theta_tnk = 65  # degree C
    return (theta_tnk - theta_wtr) * W_tnk * 10**(-3)

Example

In [ ]:
_HC_tnk = get_HC_tnk(theta_wtr = 4.587806666666667, W_tnk = 300)
_HC_tnk

In [ ]:
_L_tnk = get_L_tnk(Q_d = 100.024, HC_tnk = _HC_tnk)
_L_tnk

In [ ]:
get_L_sun_ass(L_tnk =_L_tnk, L_dash_s = [25.80175401, 33.36433708, 6.821153359, 26.69146967, 0.0, 4.831208333, 0.0])

### 6. 基準集熱量

日付$d$における1日当たりの基準集熱量$Q_{d}$

$$
\begin{equation*}
    Q_{d} = \sum_{t=0}^{23} \left( Q_{col,d,t} \times f_{hx} \times f_{s} \times t_{cp,d,t} \right)
\end{equation*}
$$

$f_{hx}$ : 給湯部の熱交換効率(-)  
$f_{s}$ : 給湯部のシステム効率(-)  
$Q_{d}$ : 日付$d$における1日当たりの基準集熱量(MJ/日)  
$Q_{col,d,t}$ : 日付$d$の時刻$t$における1時間当たりの集熱部における集熱量(MJ/h)   
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)   

給湯部の熱交換効率$f_{hx}$

$$
\begin{equation*}
    f_{hx} = 0.25
\end{equation*}
$$

給湯部のシステム効率$f_{s}$

$$
\begin{equation*}
    f_{s} = 0.85
\end{equation*}
$$

In [ ]:
def get_Q_d(Q_col, t_cp):    
    f_hx = 0.25  # -
    f_s = 0.85   # -
    return Q_col * f_hx * f_s * t_cp

Example

In [ ]:
get_Q_d(Q_col = 0.026268672, t_cp = 0.61)

### 7. 消費電力量

#### 7.1　補機の消費電力量

日付$d$における1日当たりの補機の消費電力量のうち給湯設備への付加分$E_{E,ass,aux,W,d}$

$$
\begin{equation*}
    E_{E,ass,aux,W,d} = \sum_{t=0}^{23} E_{E,ass,aux,W,d,t}
\end{equation*}
$$

$E_{E,ass,aux,W,d}$ : 日付$d$における1日当たりの補機の消費電力量のうち給湯設備への付加分(kWh/日)   
$E_{E,ass,aux,W,d,t}$ : 日付$d$の時刻$t$における1時間当たりの補機の消費電力量のうち給湯設備への付加分(kWh/h)   

日付$d$の時刻$t$における1時間当たりの補機の消費電力量のうち給湯設備への付加分$E_{E,ass,aux,W,d,t}$

$$
\begin{equation*}
    E_{E,ass,aux,W,d,t} = E_{E,fan,d,t} + E_{E,cp,d,t}
\end{equation*}
$$

$E_{E,ass,aux,W,d,t}$ : 日付$d$の時刻$t$における1時間当たりの補機の消費電力量のうち給湯設備への付加分(kWh/h)   
$E_{E,cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの消費電力量(kWh/h)   
$E_{E,fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの消費電力量(kWh/h)   

In [ ]:
def get_E_E_ass_aux_W(E_E_fan, E_E_cp):    
    return E_E_fan + E_E_cp

#### 7.2　空気搬送ファンの消費電力量

日付$d$における1日当たりの空気搬送ファンの消費電力量$E_{E,fan,d,t}$

<p style="text-indent:2em">空気搬送ファンの自立運転用太陽光発電装置を採用する場合：</p>  
$$ 
\begin{equation*}
    E_{E,fan,d,t} =0
\end{equation*}
$$

<p style="text-indent:2em">空気搬送ファンの自立運転用太陽光発電装置を採用しない場合：</p> 
<p style="text-indent:4em">$t_{cp,d,t} = 0$の場合（循環ポンプが稼働していない場合）：</p> 
$$ 
\begin{equation*}
    E_{E,fan,d,t} = 0
\end{equation*}
$$

<p style="text-indent:4em">$0 < t_{cp,d,t}$の場合（循環ポンプが稼働している場合）：</p> 
$$ 
\begin{equation*}
    E_{E,fan,d,t} =f_{SFP} \times V_{fan,d,t} \times 10^{-3}
\end{equation*}
$$

$E_{E,fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの消費電力量(kWh/h)  
$f_{SFP}$ : 空気搬送ファンの比消費電力(W/(m<sup>3</sup>/h))  
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)  
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  

空気搬送ファンの比消費電力$f_{SFP}$

<table>
    <caption>表3 空気搬送ファンの比消費電力$f_{SFP}$</caption>
    <thead style="background-color:lightgrey" >
        <tr>
            <th style="text-align:center; text-align:center; border:1px solid black;">ファンの種別</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">ファンの比消費電力$f_{SFP}$<br/>(W/(m<sup>3</sup>/h))</th>
        </tr>
    </thead>
    <tbody>    
        <tr>
            <td style="border:1px solid black; text-align:center;">ACファン</td>
            <td style="border:1px solid black; text-align:center;">0.4</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">DCファン</td>
            <td style="border:1px solid black; text-align:center;">0.2</td>
        </tr>
    </tbody> 
<table>

In [ ]:
def get_f_SFP(Fan_Type):
    return {'AC' : 0.4, 'DC' : 0.2}[Fan_Type]

In [ ]:
def get_E_E_fan(t_cp, V_fan, PV_for_Fan, Fan_Type):
    if PV_for_Fan is True:
        return 0
    elif PV_for_Fan is False:
        if t_cp == 0:
            return 0
        else:
            f_SFP = get_f_SFP(Fan_Type)  # W/(m3/h)
            return f_SFP * V_fan * 10**(-3)
    else:
        raise Exeption

#### 7.3　循環ポンプの消費電力量

日付$d$の時刻$t$における1時間当たりの循環ポンプの消費電力量$E_{E,cp,d,t}$

$$
\begin{equation*}
    E_{E,cp,d,t} = P_{cp} \times t_{cp,d,t} \times 10^{-3}
\end{equation*}
$$

$E_{E,cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの消費電力量(kWh/h)   
$P_{cp}$ : 循環ポンプの消費電力(W)   
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)  

循環ポンプの消費電力$P_{cp}$

<table>
    <caption>表4 循環ポンプの消費電力$P_{cp}$</caption>
    <thead style="background-color:lightgrey" >
        <tr>
            <th style="text-align:center; text-align:center; border:1px solid black;">循環ポンプの自立運転用太陽光発電装置の採用の有無</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">循環ポンプの消費電力$P_{cp}$<br/>(W)</th>
        </tr>
    </thead>
    <tbody>    
        <tr>
            <td style="border:1px solid black; text-align:center;">採用しない</td>
            <td style="border:1px solid black; text-align:center;">80</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">採用する</td>
            <td style="border:1px solid black; text-align:center;">0</td>
        </tr>
    </tbody> 
<table>

In [ ]:
def get_P_cp(PV_for_Pump):
    return {False : 80, True : 0}[PV_for_Pump]

In [ ]:
def get_E_E_pump(t_cp, PV_for_Pump):
    P_cp = get_P_cp(PV_for_Pump)  # W
    return P_cp * t_cp * 10**(-3)

Example

In [ ]:
_E_E_fan = get_E_E_fan(t_cp = 0.61, V_fan = 720, PV_for_Fan = False, Fan_Type = 'AC')
_E_E_fan

In [ ]:
_E_E_pump = get_E_E_pump(t_cp = 0.61, PV_for_Pump = False)
_E_E_pump

In [ ]:
get_E_E_ass_aux_W(E_E_fan = _E_E_fan, E_E_cp = _E_E_pump)

### 8. 空気搬送ファンおよび循環ポンプ

#### 8.1　空気搬送ファンの稼働時間

日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間$t_{fan,d,t}$  

<p style="text-indent:2em">$30 \leq \theta_{col,nonopg,d,t}$ かつ $25 < \theta_{col,opg,d,t}$の場合：</p> 
$$ 
\begin{equation*}
   t_{fan,d,t} = 1
\end{equation*}
$$

<p style="text-indent:2em">$\theta_{col,nonopg,d,t} < 30$ または $\theta_{col,opg,d,t} \leq 25$の場合：</p> 
$$ 
\begin{equation*}
   t_{fan,d,t} = 0
\end{equation*}
$$

$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{col,nonopg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱部の出口における空気温度(℃)  

In [ ]:
def get_t_fan(theta_col_opg, theta_col_nonopg):
    if 30 <= theta_col_nonopg and 25 < theta_col_opg:
        return 1
    elif theta_col_nonopg < 30 or theta_col_opg <= 25:
        return 0
    else:
        raise Exception

#### 8.2　循環ポンプの稼働時間

日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間$t_{cp,d,t}$

<p style="text-indent:2em">$0 < t_{fan,d,t}$の場合：</p> 
$$ 
\begin{equation*}
   t_{cp,d,t} = 1 \times r_{cp}
\end{equation*}
$$

<p style="text-indent:2em">$t_{fan,d,t} = 0$の場合：</p> 
$$ 
\begin{equation*}
   t_{cp,d,t} = 0
\end{equation*}
$$

$r_{cp}$ : 循環ポンプの変平均稼働率(-)  
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)  
$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)  

 循環ポンプの変平均稼働率（暖房運転と循環ポンプの稼働が同時に発生しないことを考慮するための係数）$r_{cp}$ 

<table>
    <caption>表4 循環ポンプの消費電力$P_{cp}$</caption>
    <thead style="background-color:lightgrey" >
        <tr>
            <th style="text-align:center; text-align:center; border:1px solid black;" rowspan=2 >地域区分</th>
            <th style="text-align:center; text-align:center; border:1px solid black;" colspan=5>年間日射量地域区分</th>
        </tr>
        <tr>
            <th style="text-align:center; text-align:center; border:1px solid black;">A1</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">A2</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">A3</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">A4</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">A5</th>
        </tr>
    </thead>
    <tbody>    
        <tr>
            <td style="border:1px solid black; text-align:center;">1</td>
            <td style="border:1px solid black; text-align:center;">0.41</td>
            <td style="border:1px solid black; text-align:center;">0.33</td>
            <td style="border:1px solid black; text-align:center;">0.32</td>
            <td style="border:1px solid black; text-align:center;">-</td>
            <td style="border:1px solid black; text-align:center;">-</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">2</td>
            <td style="border:1px solid black; text-align:center;">0.37</td>
            <td style="border:1px solid black; text-align:center;">0.44</td>
            <td style="border:1px solid black; text-align:center;">0.40</td>
            <td style="border:1px solid black; text-align:center;">-</td>
            <td style="border:1px solid black; text-align:center;">-</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">3</td>
            <td style="border:1px solid black; text-align:center;">0.44</td>
            <td style="border:1px solid black; text-align:center;">0.40</td>
            <td style="border:1px solid black; text-align:center;">0.35</td>
            <td style="border:1px solid black; text-align:center;">0.36</td>
            <td style="border:1px solid black; text-align:center;">0.39</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">4</td>
            <td style="border:1px solid black; text-align:center;">0.40</td>
            <td style="border:1px solid black; text-align:center;">0.45</td>
            <td style="border:1px solid black; text-align:center;">0.45</td>
            <td style="border:1px solid black; text-align:center;">0.39</td>
            <td style="border:1px solid black; text-align:center;">0.41</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">5</td>
            <td style="border:1px solid black; text-align:center;">0.50</td>
            <td style="border:1px solid black; text-align:center;">0.50</td>
            <td style="border:1px solid black; text-align:center;">0.53</td>
            <td style="border:1px solid black; text-align:center;">0.50</td>
            <td style="border:1px solid black; text-align:center;">0.47</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">6</td>
            <td style="border:1px solid black; text-align:center;">-</td>
            <td style="border:1px solid black; text-align:center;">0.67</td>
            <td style="border:1px solid black; text-align:center;">0.64</td>
            <td style="border:1px solid black; text-align:center;">0.62</td>
            <td style="border:1px solid black; text-align:center;">0.61</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">7</td>
            <td style="border:1px solid black; text-align:center;">0.78</td>
            <td style="border:1px solid black; text-align:center;">0.78</td>
            <td style="border:1px solid black; text-align:center;">0.78</td>
            <td style="border:1px solid black; text-align:center;">0.71</td>
            <td style="border:1px solid black; text-align:center;">0.74</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">8</td>
            <td style="border:1px solid black; text-align:center;">-</td>
            <td style="border:1px solid black; text-align:center;">-</td>
            <td style="border:1px solid black; text-align:center;">1.00</td>
            <td style="border:1px solid black; text-align:center;">1.00</td>
            <td style="border:1px solid black; text-align:center;">1.00</td>
        </tr>
    </tbody> 
<table>

In [ ]:
def get_r_cp(region, region_solar):
    return {
        'Region1' : {'A1' : 0.41, 'A2' : 0.33, 'A3' : 0.32, 'A4' : None, 'A5' : None},
        'Region2' : {'A1' : 0.37, 'A2' : 0.44, 'A3' : 0.40, 'A4' : None, 'A5' : None},
        'Region3' : {'A1' : 0.44, 'A2' : 0.40, 'A3' : 0.35, 'A4' : 0.36, 'A5' : 0.39},
        'Region4' : {'A1' : 0.40, 'A2' : 0.45, 'A3' : 0.45, 'A4' : 0.39, 'A5' : 0.41},
        'Region5' : {'A1' : 0.50, 'A2' : 0.50, 'A3' : 0.53, 'A4' : 0.50, 'A5' : 0.47},
        'Region6' : {'A1' : None, 'A2' : 0.67, 'A3' : 0.64, 'A4' : 0.62, 'A5' : 0.61},
        'Region7' : {'A1' : 0.78, 'A2' : 0.78, 'A3' : 0.78, 'A4' : 0.71, 'A5' : 0.74},
        'Region8' : {'A1' : None, 'A2' : None, 'A3' : 1.00, 'A4' : 1.00, 'A5' : 1.00}
    }[region][region_solar]

In [ ]:
def get_t_cp(t_fan, region, region_solar):
    if 0 < t_fan:
        r_cp = get_r_cp(region, region_solar)
        if r_cp is None:
            raise Exception('Solar system can\'t be applied in this region' )
        else:
            return 1 * r_cp
    elif t_fan == 0:
        return 0
    else:
        raise Exeption

#### 8.3　空気搬送ファンの風量

日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量$V_{fan,d,t}$

$$ 
\begin{equation*}
   V_{fan,d,t} = V_{fan,P0} \times t_{fan,d,t}
\end{equation*}
$$

$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)  
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  
$V_{fan,P0}$ : 空気搬送ファンの送風機特性曲線において機外静圧をゼロとした時の空気搬送ファンの風量(m<sup>3</sup>/h)  

In [ ]:
def get_V_fan(t_fan, V_fan_P0):
    return V_fan_P0 * t_fan

Example

In [ ]:
_t_fan = get_t_fan(theta_col_opg = 31.06, theta_col_nonopg = 53.64)
_t_fan

In [ ]:
get_t_cp(t_fan = _t_fan, region = 'Region6', region_solar = 'A5')

In [ ]:
get_V_fan(t_fan = _t_fan, V_fan_P0 = 720)

### 9. 集熱部

#### 9.1　集熱量

日付$d$の時刻$t$における1時間当たりの集熱部における集熱量$Q_{col,d,t}$

$$ 
\begin{equation*}
   Q_{col,d,t} = \rho_{air} \times c_{P_{air}} \times V_{fan,d,t} \times \left( \theta_{col,opg,d,t} - \theta_{ex,d,t} \right) \times 10^{-3}
\end{equation*}
$$

$c_{P_{air}}$ : 空気の比熱(kJ/(kg$\cdot$K))  
$Q_{col,d,t}$ : 空日付$d$の時刻$t$における1時間当たりの集熱部における集熱量(MJ/h)  
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{ex,d,t}$ : 日付$d$の時刻$t$における外気温度(℃)  
$\rho_{air}$ : 空気の密度(kg/m<sup>3</sup>)  

空気の密度$\rho_{air}$

$$ 
\begin{equation*}
   \rho_{air} = 1.2
\end{equation*}
$$

空気の比熱$c_{P_{air}}$

$$ 
\begin{equation*}
   c_{P_{air}} = 1.006
\end{equation*}
$$

In [ ]:
def get_Q_col(V_fan, theta_col_opg, theta_ex):
    rho_air = 1.2    # kg/m3
    c_P_air = 1.006  # kJ/(kg K)
    return rho_air * c_P_air * V_fan * (theta_col_opg - theta_ex) *10**(-3)

#### 9.2　空気搬送ファン停止時の集熱部の出口における空気温度

日付$d$の時刻$t$における空気搬送ファン停止時の集熱部の出口における空気温度$\theta_{col,nonopg,d,t}$

$$ 
\begin{equation*}
   \theta_{col,nonopg,d,t} = \frac{\displaystyle \sum_{j=1}^{n} \left( \theta_{col,nonopg,j,d,t} \times V_{col,j,d,t} \right)}{\displaystyle \sum_{j=1}^{n} V_{col,j,d,t}}
\end{equation*}
$$

$n$ : 集熱器群の数(-)  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(m<sup>3</sup>/h)  
$\theta_{col,nonopg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱部の出口における空気温度(℃)  
$\theta_{col,nonopg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度(℃)  

In [ ]:
def get_theta_col_nonopg(theta_col_nonopg_s, V_col_s, n_col):
    return sum(theta_col_nonopg_s[j1] * V_col_s[j1] for j1 in range(0,n_col,1)) / sum(V_col_s[j2] for j2 in range(0,n_col,1))

#### 9.3　空気搬送ファン稼働時の集熱部の出口における空気温度

日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度$\theta_{col,opg,d,t}$

$$ 
\begin{equation*}
   \theta_{col,opg,d,t} = \frac{\displaystyle \sum_{j=1}^{n} \left( \theta_{col,opg,j,d,t} \times V_{col,j,d,t} \right)}{\displaystyle \sum_{j=1}^{n} V_{col,j,d,t}}
\end{equation*}
$$

$n$ : 集熱器群の数(-)  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(-)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{col,opg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱器群$j$の出口における空気温度(℃)  

In [ ]:
def get_theta_col_opg(theta_col_opg_s, V_col_s, n_col):
    return sum(theta_col_opg_s[j] * V_col_s[j] for j in range(0,n_col,1)) / sum(V_col_s[j] for j in range(0,n_col,1))

#### 9.4　空気搬送ファン停止時の集熱器群の出口における空気温度

日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度$\theta_{col,nonopg,j,d,t}$

$$ 
\begin{equation*}
   \theta_{col,nonopg,j,d,t} = \frac{d_{0,j}}{d_{1,j}} \times I_{s,j,d,t} + \theta_{ex,d,t}
\end{equation*}
$$

$d_{0,j}$ : 集熱器群$j$を構成する集熱器の集熱効率特性線図一次近似式の切片(-)  
$d_{1,j}$ : 集熱器群$j$を構成する集熱器の集熱効率特性線図一次近似式の傾き(W/(m<sup>2</sup>$\cdot$K))  
$I_{s,j,d,t}$ : 日付$d$の時刻$t$における集熱器群$j$の単位面積当たりの平均日射量(W/m<sup>2</sup>)  
$\theta_{ex,d,t}$ : 日付$d$の時刻$t$における外気温度(℃)  
$\theta_{col,nonopg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度(℃)  

In [ ]:
def get_theta_col_nonopg_j(I_s, theta_ex, d_0, d_1):
    return d_0 / d_1 * I_s + theta_ex

#### 9.5　空気搬送ファン稼働時の集熱器群の出口における空気温度

日付$d$の時刻$t$における空気搬送ファン稼働時の集熱器群$j$の出口における空気温度$\theta_{col,opg,j,d,t}$

$$ 
\begin{equation*}
   \theta_{col,opg,j,d,t} = \theta_{col,nonopg,j,d,t} + \left( \theta_{ex,d,t} - \theta_{col,nonopg,j,d,t} \right) \times e^{- \frac{U_{c,j} \times A_{j}}{c_{P_{air}} \times \rho_{air} \times V_{col,j,d,t} \div 3600 \times 10^{3}}}
\end{equation*}
$$

$A_{j}$ : 集熱器群$j$の面積(m<sup>2</sup>)  
$c_{P_{air}}$ : 空気の比熱(kJ/(kg$\cdot$K))  
$U_{c,j}$ : 集熱器群$j$を構成する集熱器の集熱器総合熱損失係数(W/(m<sup>2</sup>$\cdot$K))  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(m<sup>3</sup>/h)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{col,nonopg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度(℃)  
$\theta_{ex,d,t}$ : 日付$d$の時刻$t$における外気温度(℃)  
$\rho_{air}$ : 空気の密度(kg/m<sup>3</sup>)  

In [ ]:
def get_theta_col_opg_j(theta_col_nonopg, U_c, V_col, theta_ex, A):
    rho_air = 1.2    # kg/m3
    c_P_air = 1.006  # kJ/(kg K)    

    return theta_col_nonopg + (theta_ex - theta_col_nonopg) * \
            math.exp(-U_c * A / (c_P_air * rho_air * V_col / 3600 * 10**(3)))

日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量$V_{col,j,d,t}$

$$ 
\begin{equation*}
   V_{col,j,d,t} = V_{fan,P0} \times \frac{A_{j}}{\displaystyle \sum_{j=1}^{n} A_{j}}
\end{equation*}
$$

$A_{j}$ : 集熱器群$j$の面積(m<sup>2</sup>)  
$n$ : 集熱器群の数(-)  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(m<sup>3</sup>/h)  
$V_{fan,P0}$ : 空気搬送ファンの送風機特性曲線において機外静圧をゼロとした時の空気搬送ファンの風量(m<sup>3</sup>/h)  

In [ ]:
def get_sum_A(A_s):
    return sum(A for A in A_s)

In [ ]:
def get_V_col_j(V_fan_P0, A, sum_A):
    return V_fan_P0 * A / sum_A

集熱器群$j$を構成する集熱器の集熱器総合熱損失係数$U_{c,j}$

$$ 
\begin{equation*}
   U_{c,j} = -c_{P_{air}} \times \dot{m}_{fan,test,j} \times 10^{3} \times ln \left( 1 - \frac{1}{c_{P_{air}} \times \dot{m}_{fan,test,j} \times 10^{3}} \times d_{1,j}  \right)
\end{equation*}
$$

$c_{P_{air}}$ : 空気の比熱(kJ/(kg$\cdot$K))  
$d_{1,j}$ : 集熱器群$j$を構成する集熱器の集熱効率特性線図一次近似式の傾き(W/(m<sup>2</sup>$\cdot$K))  
$\dot{m}_{fan,test,j}$ : 集熱器群$j$を構成する集熱器の集熱性能試験時における単位面積当たりの空気の質量流量(kg/(s$\cdot$m<sup>2</sup>))  
$U_{c,j}$ : 集熱器群$j$を構成する集熱器の集熱器総合熱損失係数(W/(m<sup>2</sup>$\cdot$K))  

In [ ]:
def get_U_c_j(m_dot_fan_test, d_1):
    c_P_air = 1.006  # kJ/(kg K)  
    return -c_P_air * m_dot_fan_test * 10**3 * math.log(1 - 1 / (c_P_air * m_dot_fan_test * 10**3) * d_1)

Example

In [ ]:
_theta_col_nonopg_s = [get_theta_col_nonopg_j(I_s = [748.495, 748.495][j], theta_ex = 8.5, 
                                             d_0 = [0.1, 0.468][j], d_1 = [2.0, 5.816][j]) for j in range(0,2,1)]
_theta_col_nonopg_s

In [ ]:
_U_c_s= [get_U_c_j(m_dot_fan_test = [0.0107, 0.0107][j], d_1 = [2.0, 5.816][j]) for j in range(0,2,1)]
_U_c_s

In [ ]:
_sum_A = get_sum_A(A_s = [15, 15])
_sum_A

In [ ]:
_V_col_s = [get_V_col_j(V_fan_P0 = 720, A = [15, 15][j], sum_A = _sum_A)for j in range(0,2,1)]
_V_col_s

In [ ]:
_theta_col_opg_s = [get_theta_col_opg_j(theta_col_nonopg = _theta_col_nonopg_s[j], U_c = _U_c_s[j], V_col =_V_col_s[j], 
                                       theta_ex = 8.5, A =  [15, 15][j])for j in range(0,2,1)]
_theta_col_opg_s

In [ ]:
get_theta_col_nonopg(theta_col_nonopg_s= _theta_col_nonopg_s, V_col_s = _V_col_s, n_col = 2)

In [ ]:
get_theta_col_opg(theta_col_opg_s= _theta_col_opg_s, V_col_s = _V_col_s, n_col = 2)

In [ ]:
get_Q_col(V_fan = 720, theta_col_opg = 32.46, theta_ex = 8.5)

### Integrated Function

#### Input Data

spec = {  

    'region'                             : 'Region1', 'Region2', 'Region3', 'Region4', 'Region5', 'Region6', 'Region7', 'Region8' as string  
    'region_solar'                       : 'A1', 'A2', 'A3', 'A4', 'A5' as string  
    
    'number of collector unit'           : 0,  
    'azimuth of collector units'         :  1: less then 15 degree to east or west from due south,   
                                            2:  15 degree or more and less than  45 degree to east from due south,  
                                            3:  45 degree or more and less than  75 degree to east from due south,  
                                            4:  75 degree or more and less than 105 degree to east from due south,  
                                            5: 105 degree or more and less than 135 degree to east from due south,  
                                            6: 135 degree or more and less than 165 degree to east from due south,  
                                            7:  15 degree or less from due north,    
                                            8: 135 degree or more and less than 165 degree to west from due south,  
                                            9: 105 degree or more and less than 135 degree to west from due south,  
                                           10:  75 degree or more and less than 105 degree to west from due south,  
                                           11:  45 degree or more and less than  75 degree to west from due south,  
                                           12:  15 degree or more and less than  45 degree to west from due south  
    'PV for fan'                         : 1 : Not equipped, 2 : Equipped,  as string    
    'fan type'                           : 1 : 'DC', 2 : 'AC',    as string  
    'PV for circulation pump'            : 1 : Not equipped, 2 : Equipped,  as string  
    'tank size'                          : 0,   # (L)  
    'fan air flow rate'                  : 0.0, # at the external static pressure of zero (m3/h)  

    'area of collector unit'             : 0.0  # (m2)  
    'angle of collector units'           : 1: 0 degree (horizontal), 2: 10 degree, 3: 20 degree, 4: 30 degree, 5: 40 degree,   
                                           6: 50 degree, 7: 60 degree, 8: 70 degree, 9: 80 degree, 10: 90 degree (vertical)
    'inclination of characteristic line' : 0.0  # (-)  
    'intercept of characteristic line'   : 0.0  # (W/(m2K))  
    'mass flow rate'                     : 0.000  # when testing according to JIS (kg/(sm2))  
}

Example

Specification / 仕様

In [ ]:
spec = {  
    'region'                             : 'Region6',  
    'region_solar'                       : 'A5',  

    'number of collector unit'           : 2,  
    'azimuth of collector units'         : '1',  
    'PV for fan'                         : '1',  
    'fan type'                           : '2',  
    'PV for circulation pump'            : '1',  
    'tank size'                          : 300,  
    'fan air flow rate'                  : 720.0,  

    'area of collector unit'             : [15.0,   15.0], 
    'angle of collector units'           : ['4',    '4'],
    'inclination of characteristic line' : [0.1,    0.468],  
    'intercept of characteristic line'   : [2.0,    5.816],  
    'mass flow rate'                     : [0.0107, 0.0107] 
}

Hot water load / 給湯負荷

In [ ]:
LWT.get_L_dash_W_s('1/1')

Daily Average Water Temperature / 日平均給水温度

In [ ]:
LWT.get_theta_wtr('1/1')

#### Hourly Calculation / 時間計算

In [ ]:
def get_hourly_Q_d_and_E_E(spec, day, hour):
    
    # region
    region = spec['region']
    region_solar = spec['region_solar']
    
    # number of collector unit
    n_col = spec['number of collector unit']
    
    # azimuth of collector units
    azimuth_col = spec['azimuth of collector units']
    
    # PV for fan / pump
    PV_for_Fan = {'1' : False, '2' : True}[spec['PV for fan']]
    Fan_Type = {'1' : 'DA', '2' : 'AC'}[spec['fan type']]
    PV_for_Pump = {'1' : False, '2' : True}[ spec['PV for circulation pump']]
    
    #  fan air flow rate
    V_fan_P0 = spec['fan air flow rate']
    
    # area of collector unit
    A_s = spec['area of collector unit']
    # angle of collector units
    angle_col = spec['angle of collector units']
    # inclination of characteristic line
    d_0_s = spec['inclination of characteristic line']
    # intercept of characteristic line
    d_1_s = spec['intercept of characteristic line']
    # mass flow rate
    m_dot_fan_test_s = spec['mass flow rate']
    
    # outdoor temperature
    theta_ex = EC.get_temperature(region, day, hour)
    
    # solar
    P_alpha = SR.get_P_alpha(azimuth_col)
    P_beta = [SR.get_P_beta(b) for b in angle_col]
    I_s_s = [SR.get_I_s(region, region_solar,day, hour, P_alpha, P_beta[j]) for j in range(0,n_col,1)]

    # air temperature at outlet of each collector unit when non-operating
    theta_col_nonopg_s = [get_theta_col_nonopg_j(I_s_s[j], theta_ex, d_0_s[j], d_1_s[j]) for j in range(0,n_col,1)]
    # air temperature at outlet of each collector unit when operating
    theta_col_opg_s = [get_theta_col_opg_j(theta_col_nonopg_s[j], 
                                          [get_U_c_j(m_dot_fan_test_s[jj], d_1_s[jj]) for jj in range(0,n_col,1)][j], 
                                          [get_V_col_j(V_fan_P0, A_s[jj], get_sum_A(A_s)) for jj in range(0,n_col,1)][j], 
                                          theta_ex, A_s[j]) for j in range(0,n_col,1)]
    # average air temperature at outlet of collector units when non-operating
    theta_col_nonopg = get_theta_col_nonopg(theta_col_nonopg_s, 
                                            [get_V_col_j(V_fan_P0, A_s[j], get_sum_A(A_s)) for j in range(0,n_col,1)], n_col) 
    # average air temperature at outlet of collector units when operating
    theta_col_opg = get_theta_col_opg(theta_col_opg_s, 
                                      [get_V_col_j(V_fan_P0, A_s[j], get_sum_A(A_s)) for j in range(0,n_col,1)], n_col)

    # fan perating duration
    t_fan = get_t_fan(theta_col_opg, theta_col_nonopg)
    # pump perating duration
    t_cp = get_t_cp (get_t_fan(theta_col_opg, theta_col_nonopg), region, region_solar)
    # fan air flow rate
    V_fan = get_V_fan(t_fan, V_fan_P0)
    
    # amount of heat collected by the solar heat collector
    Q_col = get_Q_col(V_fan, theta_col_opg, theta_ex)
    
    # utilizable heat for water heating 
    Q_d = get_Q_d(Q_col, t_cp)

    # power consumption of fan
    E_E_fan  = get_E_E_fan (t_cp, V_fan, PV_for_Fan, Fan_Type)
    # power consumption of pump
    E_E_pump = get_E_E_pump(t_cp, PV_for_Pump)
    # power consumption of auxiliary machinery
    E_E_ass_aux_W = get_E_E_ass_aux_W(E_E_fan = E_E_fan, E_E_cp = E_E_pump)
                     
    return Q_col, Q_d, E_E_ass_aux_W

Example

In [ ]:
get_hourly_Q_d_and_E_E(spec = spec,  day = '1/1', hour = 12)

#### Daily Calculation / 日計算

In [ ]:
def get_daily_Q_col(spec, day):   
    return sum([get_hourly_Q_d_and_E_E(spec, day, hour) [0] for hour in range(0, 24, 1)])

In [ ]:
def get_daily_Q_d(spec, day):   
    return sum([get_hourly_Q_d_and_E_E(spec, day, hour) [1] for hour in range(0, 24, 1)])

In [ ]:
def get_daily_E_E(spec, day):   
    return sum([get_hourly_Q_d_and_E_E(spec, day, hour) [2] for hour in range(0, 24, 1)])

In [ ]:
def get_daily_L_sun_ass(spec, Q_d, day):
    
    # tank size
    W_tnk = spec['tank size']
    
    return get_L_sun_ass(get_L_tnk(Q_d, get_HC_tnk(LWT.get_theta_wtr(day), W_tnk)), LWT.get_L_dash_W_s(day))

Example

In [ ]:
get_daily_E_E(spec = spec, day = '1/1')

In [ ]:
get_daily_Q_col(spec = spec, day = '1/1')

In [ ]:
get_daily_Q_d(spec = spec, day = '1/1')

In [ ]:
get_daily_L_sun_ass(spec = spec, Q_d = 12.965698043346025, day = '1/1')

In [ ]:
plt.figure(figsize = (16, 3), dpi=80 )
plt.subplot2grid((1, 1), (0, 0))
plt.title('Calcuation Example')

x_values = [x for x in range(0, 365, 1)]
y_values = [get_daily_Q_col(spec, day) for day in EC.get_Annual_Days()]

plt.plot(x_values, y_values, label='Q_col',)
        
ax=plt.gca()
ax.set_xlabel('day')
ax.set_xticks([0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334])
ax.set_xticklabels(['1/1', '2/1', '3/1', '4/1', '5/1', '6/1', '7/1', '8/1', '9/1', '10/1', '11/1', '12/1'], minor=False)
ax.set_xbound(lower=0, upper=365)

ax.set_ylabel('Q_col(MJ/day)')
ax.set_yticks([y for y in range(0, 210, 20)], minor=False)
ax.set_ybound(lower=0, upper=200)

ax.grid(which='major', color='lightgray', alpha=0.7, linestyle='-')     
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), borderaxespad=0.5, frameon=True)

plt.show()

In [ ]:
plt.figure(figsize = (16, 3), dpi=80 )
plt.subplot2grid((1, 1), (0, 0))
plt.title('Calcuation Example')

x_values = [x for x in range(0, 365, 1)]
y_values = [get_daily_L_sun_ass(spec, get_daily_Q_d(spec, day), day) for day in EC.get_Annual_Days()]

plt.plot(x_values, y_values, label='L_sun_ass',)

ax=plt.gca()
ax.set_xlabel('day')
ax.set_xticks([0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334])
ax.set_xticklabels(['1/1', '2/1', '3/1', '4/1', '5/1', '6/1', '7/1', '8/1', '9/1', '10/1', '11/1', '12/1'], minor=False)
ax.set_xbound(lower=0, upper=365)

ax.set_ylabel('L_sun_ass(MJ/day)')
ax.set_yticks([y for y in range(0, 25, 5)], minor=False)
ax.set_ybound(lower=0, upper=20)

ax.grid(which='major', color='lightgray', alpha=0.7, linestyle='-')     
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), borderaxespad=0.5, frameon=True)

plt.show()

#### Annual Calculation / 年間計算

Example

In [ ]:
annual_E_E_acc_aux_W = sum([get_daily_E_E(spec, day) for day in EC.get_Annual_Days()])
print('Power consumption of auxiliary machinery : ' + str(annual_E_E_acc_aux_W) + ' kWh/year')

In [ ]:
annual_Q_d = sum([get_daily_Q_d(spec, day) for day in EC.get_Annual_Days()]) /1000
print('Annal amount of heat collected by the solar heat collector : ' + str(annual_Q_d) + ' GJ/year')

In [ ]:
annual_L_sun_ass = sum([get_daily_L_sun_ass(spec, get_daily_Q_d(spec, day), day) for day in EC.get_Annual_Days()]) /1000
print('Annal hot water load after substracting amount of heat collected by the solar heat collector : ' + str(annual_L_sun_ass) + ' GJ/year')

In [ ]:
annual_HW_Load = sum([sum([LWT.get_L_dash_W_s(day)[i] for i in range(0,6)]) for day in EC.get_Annual_Days()]) /1000
print('Annal hot water load : ' + str(annual_HW_Load) + ' GJ/year')

In [ ]:
print('Redution ratio : ' + str(annual_L_sun_ass / annual_HW_Load * 100) + ' %')